## Постановка задачи
Загрузим подготовленные данные из HDF5. Разделим данные на обучающие и проверочные и построим модель опорных векторов для типа облака Fish.

Проведем оценку качества предсказания по F1 и коэффициенту сходства.

Данные:
* https://video.ittensive.com/machine-learning/clouds/clouds.data.h5 (959 Мб)

Соревнование: https://www.kaggle.com/c/understanding_cloud_organization/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

### Загрузка данных

In [2]:
clouds = pd.read_hdf("clouds.data.h5")
print (clouds.head())

    0   1    2    3    4    5    6    7    8    9  ...  183745  183746  \
0   0   0    0    0    0    0    0    0    0    0  ...     102     113   
1  71  42   31   38   48   68   70   46   24   21  ...      38      57   
2  97  98  100  102  105  107  109  110  111  112  ...       0       0   
3  87  86   89   93   93   88   86   86   81   81  ...     165     164   
4  18  19   21   21   20   19   17   16   14   15  ...     125     137   

   183747  183748  183749        Image  \
0     102      96     116  0011165.jpg   
1      82     110     133  002be4f.jpg   
2       0       0       0  0031ae9.jpg   
3     108      70      72  0035239.jpg   
4      92      59      48  003994e.jpg   

                                                Fish  \
0  264918 937 266318 937 267718 937 269118 937 27...   
1  233813 878 235213 878 236613 878 238010 881 23...   
2  3510 690 4910 690 6310 690 7710 690 9110 690 1...   
3                                                NaN   
4  2367966 18 2367985 

Оставим только данные по Fish, на них обучим модель

In [3]:
clouds.drop(labels=["Image", "Flower", "Gravel", "Sugar"],
           axis=1, inplace=True)

### Разделение данных
Разделим всю выборку на 2 части случайным образом: 80% - для обучения модели, 20% - для проверки точности модели.

In [4]:
clouds_train, clouds_test = train_test_split(clouds, test_size=0.2)
clouds_train = pd.DataFrame(clouds_train)
clouds_test = pd.DataFrame(clouds_test)
del clouds
print (clouds_train.head())

        0    1   2   3   4   5   6   7   8   9  ...  183741  183742  183743  \
3163   22   22  22  22  22  22  22  22  22  22  ...      24      21      25   
5476   66   66  57  58  79  90  77  64  74  48  ...     167     174     177   
3203  147  105  60  36  17  32  59  39  39  43  ...      57      63      61   
2198   41   53  43  31  41  48  45  49  67  91  ...     189     184     182   
3555   87   89  91  93  94  94  93  93  90  94  ...      55      56      59   

      183744  183745  183746  183747  183748  183749  \
3163      42      76     148     176     163     199   
5476     201     186     185     173     154     151   
3203      58      69      80     142     199     201   
2198     176     157     151     171     182     159   
3555      62      66      65      64      63      63   

                                                   Fish  
3163  791043 817 792443 817 793843 817 795243 817 79...  
5476  294857 530 296257 530 297657 530 299057 530 30...  
3203          

### Метод опорных векторов
Последовательно рассчитываем коэффициенты для пакетов по 100 изображений, иначе может не поместиться в оперативную память.

Используем warm_start=True, чтобы переиспользовать предыдущие параметры.

In [5]:
y = clouds_train["Fish"].notnull().astype("int8")
x = pd.DataFrame(clouds_train).drop(labels=["Fish"], axis=1)
model = SGDClassifier(loss="log", warm_start=True)

In [6]:
for i in range (len(clouds_train)//100):
    model.partial_fit(x[i:i+100], y[i:i+100], classes=[0,1])

In [7]:
del x
del y

### Средняя область облаков
В качестве локализации облаков на изображении возьмем среднюю область по обучающей выборке

In [8]:
image_x = 2100
image_x_4 = image_x//4
image_y = 1400
image_y_4 = image_y//4

In [9]:
def locate_rectangle (a):
    vals = [int(i) for i in a[0].split(" ")]
    x = vals[0]//image_y
    y = vals[0]%image_y
    width = (image_x + (vals[-2] + vals[-1])//image_y - x)%image_x
    height = (vals[-2] + vals[-1])%image_y - y
    return [x, y, width, height]

In [10]:
areas = pd.DataFrame(clouds_train["Fish"].copy().dropna(axis=0))
areas = areas.apply(locate_rectangle, axis=1, result_type="expand")
coords = np.array(areas.mean()//4)
print (coords)

[ 94. 103. 332. 111.]


In [12]:
sgd_mask = np.zeros(image_x_4*image_y_4,
                   dtype="uint8").reshape(image_x_4, image_y_4)
for x in range(image_x_4):
    for y in range(image_y_4):
        if (x >= coords[0] and x<= (coords[0] + coords[3]) and
           y >= coords[1] and y<= (coords[1] + coords[3])):
            sgd_mask[x][y] = 1
sgd_mask = sgd_mask.reshape(image_x_4*image_y_4)
print (sgd_mask.sum())

12544


### Предсказание значений

In [19]:
result = pd.DataFrame({"EncodedPixels":clouds_test["Fish"],
            "Is_Fish": clouds_test["Fish"].notnull().astype("int8")})
result["target"] = model.predict(pd.DataFrame(clouds_test).drop(labels=["Fish"],
                                                               axis=1))
print (result.head(10))

                                          EncodedPixels  Is_Fish  target
2447  21087 398 22487 398 23887 398 25287 398 26687 ...        1       1
2192  31262 337 32662 337 34062 337 35462 337 36862 ...        1       0
1741                                                NaN        0       0
4343  189766 312 191166 312 192566 312 193966 312 19...        1       0
2628                                                NaN        0       0
855                                                 NaN        0       1
3693  519 458 1919 458 3319 458 4719 458 6119 458 75...        1       1
1826  47607 546 49007 546 50407 546 51807 546 53207 ...        1       1
1632  2208618 530 2210018 530 2211418 530 2212818 53...        1       0
1092  777 492 2177 492 3577 492 4977 492 6377 492 77...        1       1


### Оценка точности предсказания: F1
Точность = TruePositive / (TruePositive + FalsePositive)

Полнота = TruePositive / (TruePositive + FalseNegative)

F1 = 2 * Полнота * Точность / (Полнота + Точность)

In [16]:
print ("Опорные векторы:",
      round(f1_score(result["Is_Fish"], result["target"]), 3))
print ("Нет облаков:",
      round(f1_score(result["Is_Fish"], np.ones(len(result))), 3))

Опорные векторы: 0.512
Нет облаков: 0.671


### Оценка по Дайсу
Для каждого изображения и каждой фигуры считается пересечение площади обнаруженной фигуры (X) с ее реальной площадью (Y) по формуле:
\begin{equation}
Dice = \frac{2∗|X∩Y|}{|X|+|Y|}
\end{equation}
Если и X, и Y равны 0, то оценка принимается равной 1. Оценка берется как среднее по всем фигурам.

Пока будем считать, что при определении типа облака на изображении, оно целиком размещено на фотографии: т.е. область облака - это все изображение.

Дополнительно посчитаем точность предсказания, что на фотографиях вообще нет облаков нужного типа.

In [23]:
image_x = 525
image_y = 350
def mask_rate (a, x, y):
    b = a//1400 + 0.0
    return np.round(x*(b*x//2100) + y*(a%1400)//1400).astype("uint32")

def calc_mask (px, x=image_x, y=image_y):
    p = np.array([int(n) for n in px.split(" ")]).reshape(-1, 2)
    mask = np.zeros(y*x, dtype="uint8")
    for i, l in p:
        mask[mask_rate(i, x, y) - 1:mask_rate(l + i, x, y)] = 1
    return mask.reshape(y,x).transpose()

def calc_dice (x):
    dice = 0
    px = x["EncodedPixels"]
    if px != px and x["target"] == 0:
        dice = 1
    elif px == px and x["target"] == 1:
        mask = calc_mask(px).flatten()
#        target = sgd_mask
        target = np.ones(image_x*image_y, dtype="uint8")
        dice = 2*np.sum(target[mask==1])/(np.sum(target)+np.sum(mask))
    return dice

In [24]:
dice = result.apply(calc_dice, axis=1, result_type="expand")
print ("Опорные векторы, Fish:", round(dice.mean(), 3))
print ("Нет облаков, Fish:",
      round(len(result[result["Is_Fish"]==0])/len(result), 3))

Опорные векторы, Fish: 0.337
Нет облаков, Fish: 0.495
